In [ ]:
!pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00


In [ ]:
!!pip install -q langchain_openai faiss-cpu

['\x1b[?25l   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/55.3 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b55.3/55.3 kB\x1b \x1b2.5 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25h\x1b[?25l   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/30.7 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K   \x1b━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b1.0/30.7 MB\x1b \x1b28.7 MB/s\x1b eta \x1b0:00:02\x1b',
 '\x1b[2K   \x1b━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b1.4/30.7 MB\x1b \x1b17.8 MB/s\x1b eta \x1b0:00:02\x1b',
 '\x1b[2K   \x1b━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b2.4/30.7 MB\x1b \x1b18.3 MB/s\x1b eta \x1b0:00:02\x1b',
 '\x1b[2K   \x1b━━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b3.3/30.7 MB\x1b \x1b18.6 MB/s\x1b eta \x1b0:00:02\x1b',
 '\x1b[2K   \x1b━━━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b4.1/30.7 MB

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
from google.colab import userdata
api_key=userdata.get('openai_key')

In [ ]:
from google.colab import userdata
AZURE_API_KEY=userdata.get('AZURE_API_KEY')
AZURE_ENDPOINT=userdata.get('AZURE_ENDPOINT')

In [ ]:
llm = AzureChatOpenAI(
    azure_endpoint=AZURE_ENDPOINT,
    azure_deployment="gpt-4o",
    api_version="2023-09-01-preview",
    api_key=AZURE_API_KEY,
    temperature=0.3
)


In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import json
import time
import random
diseases = [
    "Diabetes Mellitus", "Hypertension", "Alzheimer's Disease", "Parkinson's Disease",
    "Tuberculosis", "Pneumonia", "Influenza"
]
all_conversations = []

for disease in diseases:
    agents = {
        "Dr. Smith": f"Dr. Smith is an experienced specialist in {disease}, explaining its symptoms, causes, and treatment options in detail.",
        "Dr. Patel": f"Dr. Patel is an expert in treating and managing {disease}, focusing on long-term care, medications, and lifestyle changes.",
        "Dr. Lee": f"Dr. Lee is a researcher specializing in {disease}, discussing the latest advancements, clinical trials, and future treatments.",
        "James": f"James has been diagnosed with {disease}. He only asks detailed questions about symptoms, treatments, medications, and prognosis. He does not provide medical knowledge or commentary."
    }

    conversation_history = [
        HumanMessage(content=f"You have been diagnosed with {disease}. Start by asking the doctor specialised in treating {disease} basic questions about its symptoms and treatment options.", name="System")
    ]


    def generate_message(speaker, history):
      sanitized_speaker = speaker.replace(" ", "_").replace(".", "").lower()

      system_prompt = (
          f"{agents[speaker]} Keep responses well-structured, factual, and educational. "
          f"Maintain a natural, professional tone. Avoid overusing the listener's name unnecessarily, "
          f"or repeating phrases like 'let’s continue,' 'sure,' or 'of course.' "
          f"Answer directly and naturally, responding to the previous message without unnecessary transitions. "
          f"If you are a specialist, always provide a complete answer without asking James what he wants to focus on. "
          f"You are a top expert in {disease}, and your goal is to share your knowledge concisely and understandably. "
          f"Your answers should be **max 80 words**, written in simple paragraphs (not bullet points). "
          f"If you are James, your only role is to ask one specific, medically detailed question per turn. "
          f"James **must not provide medical explanations**—he only asks patient-related questions. "
          f"James must not ask doctors to clarify; instead, he must ask a direct follow-up question. "
          f"For example, instead of saying 'Can you specify your question?' James should ask 'How does insulin resistance develop in diabetes?' or 'What are the risks of uncontrolled hypertension over time?'"
      )

      print(f"\n🔍 DEBUG: Preparing request for {sanitized_speaker}...")


      start_time = time.time()
      try:
          print(f"🔍 DEBUG: Sending request to Azure OpenAI for {sanitized_speaker}...")

          response = llm([
              SystemMessage(content=system_prompt, name="system"),
              *[
                  HumanMessage(content=msg.content, name=msg.name.replace(" ", "_").replace(".", "").lower())
                  if isinstance(msg, HumanMessage)
                  else AIMessage(content=msg.content, name=msg.name.replace(" ", "_").replace(".", "").lower())
                  for msg in history[-random.randint(3, 7):]
              ],
              HumanMessage(content="Continue the conversation.", name="james")
          ])

          response_time = time.time() - start_time

          print(f"✅ DEBUG: {sanitized_speaker} responded in {response_time:.2f} seconds")
          print(f"📝 RESPONSE: {response.content[:100]}...\n")

          return AIMessage(content=response.content, name=sanitized_speaker)

      except Exception as e:
          print(f"❌ ERROR: Failed to generate message for {sanitized_speaker}")
          print(str(e))
          return AIMessage(content="(Error: No response from LLM)", name=sanitized_speaker)


    num_messages = 25
    generated_messages = 0

    for i in range(num_messages):
        if i % 2 == 0:
            current_speaker = "James"
        else:
            current_speaker = random.choice(["Dr. Smith", "Dr. Patel", "Dr. Lee"])

        new_message = generate_message(current_speaker, conversation_history)
        conversation_history.append(new_message)
        generated_messages += 1

        if generated_messages % 10 == 0:
            print(f"✅ DEBUG: Generated {generated_messages}/{num_messages} messages for {disease}...")

        time.sleep(1)
    all_conversations.append({
        "disease": disease,
        "chat": [
            {"timestamp": time.strftime('%Y-%m-%d %H:%M:%S'), "speaker": msg.name, "message": msg.content}
            for msg in conversation_history
        ]
    })


with open("large_chat_dataset.json", "w") as f:
    json.dump(all_conversations, f, indent=4)

print("\n🎉 Large dataset saved to large_chat_dataset.json")



🔍 DEBUG: Preparing request for james...
🔍 DEBUG: Sending request to Azure OpenAI for james...


<ipython-input-6-9f263b984aaf>:49: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([


✅ DEBUG: james responded in 0.82 seconds
📝 RESPONSE: What are the common symptoms of Diabetes Mellitus?...


🔍 DEBUG: Preparing request for dr_lee...
🔍 DEBUG: Sending request to Azure OpenAI for dr_lee...
✅ DEBUG: dr_lee responded in 4.52 seconds
📝 RESPONSE: Common symptoms of Diabetes Mellitus include increased thirst (polydipsia), frequent urination (poly...


🔍 DEBUG: Preparing request for james...
🔍 DEBUG: Sending request to Azure OpenAI for james...
✅ DEBUG: james responded in 0.46 seconds
📝 RESPONSE: What are the main treatment options for Diabetes Mellitus?...


🔍 DEBUG: Preparing request for dr_patel...
🔍 DEBUG: Sending request to Azure OpenAI for dr_patel...
✅ DEBUG: dr_patel responded in 1.67 seconds
📝 RESPONSE: The main treatment options for Diabetes Mellitus depend on the type. For Type 1 diabetes, insulin th...


🔍 DEBUG: Preparing request for james...
🔍 DEBUG: Sending request to Azure OpenAI for james...
✅ DEBUG: james responded in 1.23 seconds
📝 RESPONSE: What are the po

KeyboardInterrupt: 

In [ ]:
import json


file_path = "/content/large_chat_dataset.json"

with open(file_path, "r") as f:
    chat_data = json.load(f)


documents = []
metadata = []

for entry in chat_data:
    disease = entry["disease"]
    for chat in entry["chat"]:
        text = chat["message"]
        speaker = chat["speaker"]
        timestamp = chat["timestamp"]


        doc = f"Disease: {disease}\nSpeaker: {speaker}\nMessage: {text}"
        documents.append(doc)


        metadata.append({"disease": disease, "speaker": speaker, "timestamp": timestamp})


print(f"✅ Processed {len(documents)} chat messages.")
print("🔍 Sample Document:\n", documents[3])


In [ ]:
!pip install -q openai==0.28

In [ ]:
import openai
import numpy as np
import os


openai.api_key = api_key


def get_embedding(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response["data"][0]["embedding"]


embeddings = []
for doc in documents:
    embedding = get_embedding(doc)
    embeddings.append(embedding)


embeddings = np.array(embeddings, dtype=np.float32)


print(f"✅ Generated {len(embeddings)} embeddings.")
print("🔍 Sample embedding:\n", embeddings[0][:5])


In [ ]:
import faiss

embedding_dim = 1536

index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings)

faiss.write_index(index, "faiss_chat_index")

with open("metadata.json", "w") as f:
    json.dump(metadata, f, indent=4)

print("✅ FAISS index and metadata saved successfully.")


In [ ]:
import json
import openai
import numpy as np

file_path = "/content/large_chat_dataset.json"

with open(file_path, "r") as f:
    chat_data = json.load(f)

documents = []
metadata = []

for entry in chat_data:
    disease = entry["disease"]
    for chat in entry["chat"]:
        text = chat["message"]
        speaker = chat["speaker"]
        timestamp = chat["timestamp"]


        doc = f"Disease: {disease}\nSpeaker: {speaker}\nMessage: {text}"
        documents.append(doc)


        metadata.append({"disease": disease, "speaker": speaker, "timestamp": timestamp})

with open("chat_texts.json", "w") as f:
    json.dump(documents, f, indent=4)

with open("metadata.json", "w") as f:
    json.dump(metadata, f, indent=4)

print(f"✅ Saved {len(documents)} chat messages to 'chat_texts.json'.")


In [ ]:
import faiss
import openai
import numpy as np
import json


index = faiss.read_index("faiss_chat_index")

# Load metadata
with open("metadata.json", "r") as f:
    metadata = json.load(f)


with open("chat_texts.json", "r") as f:
    documents = json.load(f)


def get_embedding(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return np.array(response["data"][0]["embedding"], dtype=np.float32)


def retrieve_similar_chats(query, top_k=4):
    query_embedding = get_embedding(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)


    retrieved_docs = []
    for idx in indices[0]:
        if idx < len(metadata):
            doc_data = metadata[idx].copy()
            doc_data["message"] = documents[idx]
            retrieved_docs.append(doc_data)

    return retrieved_docs


query = "What is cholinesterase?"
retrieved_messages = retrieve_similar_chats(query)


print("\n🔍 Top Retrieved Messages:")
for msg in retrieved_messages:
    print(f"📌 Disease: {msg['disease']} | 🩺 Speaker: {msg['speaker']}")
    print(f"🕒 Timestamp: {msg['timestamp']}")
    print(f"💬 Message: {msg['message']}\n")


In [ ]:
import faiss
import openai
import numpy as np
import json


index = faiss.read_index("faiss_chat_index")

with open("metadata.json", "r") as f:
    metadata = json.load(f)

with open("chat_texts.json", "r") as f:
    documents = json.load(f)

openai.api_key = api_key

def get_embedding(text):
    response = openai.Embedding.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return np.array(response["data"][0]["embedding"], dtype=np.float32)

def retrieve_similar_chats(query, top_k=5):
    query_embedding = get_embedding(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)


    retrieved_docs = []
    for idx in indices[0]:
        if idx < len(metadata):
            doc_data = metadata[idx].copy()
            doc_data["message"] = documents[idx]
            retrieved_docs.append(doc_data)

    return retrieved_docs


def generate_response(user_query):

    retrieved_messages = retrieve_similar_chats(user_query)


    context = "\n".join([f"{msg['speaker']}: {msg['message']}" for msg in retrieved_messages])


    system_prompt = (
    "You are a medical chatbot that provides informative, well-structured, and accurate answers based solely on the information available in the chat history. "
    "Do not introduce any external knowledge, and refrain from speculating or providing information that has not been mentioned in the conversation. "
    "If a user says they have a certain symptom , you must look into your dataset to see if you know what can be the possible cause, say CLEAR no if you have no idea, and suggest to consult a doctor either way!"
    "If a question arises that is not covered within the chat history, your response should be clear and direct: 'This information is not mentioned in the chat.' "
    "You should always use the context and background from previous messages to generate your responses, ensuring relevance and clarity, but avoid repeating messages verbatim. "
    "Maintain a helpful and educational tone, providing concise and precise answers based only on the provided conversation."
)


    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"User Question: {user_query}\n\nRetrieved Context:\n{context}"}
        ]
    )

    return response["choices"][0]["message"]["content"]


print("🩺 Medical Chatbot (Type 'exit' to quit)\n")
while True:
    user_query = input("You: ")
    if user_query.lower() == "exit":
        print("👋 Goodbye!")
        break

    bot_response = generate_response(user_query)
    print(f"💬 Chatbot: {bot_response}\n")
